__Автор__: Карпаев Алексей, ассистент кафедры информатики и вычислительной математики.

# Метод конечных разностей для численного решения ОДУ: программная реализация, ОО подход¶

## Математическая постановка задачи

В данной лекции будем рассматривать __задачу Коши для автономного ОДУ__:

$$ \frac{d u}{d t} = f\left(u \right), $$
$$ u \left( 0 \right) = u_0 $$
$$ t > 0 $$
В качестве конкретного примера возьмем __логистическое уравнение__, имеющее правую часть следующего вида:
$$
f \left( u; \alpha, R \right) = \alpha \cdot u \cdot \left( 1 - \frac{u}{R} \right)
$$

Уравнение описывает динамику популяции с учетом влияния факторов окружающей среды, ограничивающих экспоненциальный рост населения со временем.

## Процесс численного решения уравнения
Производим дискретизацию расчетной области: вводим равномерную сетку с шагом $\Delta t = \frac{T}{N}$ c числом отрезков разбиения $N$. Для нахождения численного решения будем рассматривать следующий набор численных методов:
* Явный метод Эйлера 1-го порядка точности
* Метод Эйлера с пересчетом 2-го порядка точности
* Один из явных методов Рунге-Кутты 3-го порядка точности
* Один из явных методов Рунге-Кутты 4-го порядка точности
* Неявный метод трапеций 2-го порядка точности

Все методы являются __одношаговыми__ и относятся к семейству __методов Рунге-Кутты__.

## Программная реализация
### Процедурный подход

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', size=20)

In [ ]:
# параметры сетки и уравнения
numBlocks = 10
tEnd = 40.
dt = float(tEnd)/float(numBlocks)
initialValue = 0.1

In [1]:
class LogisticRightHandSide:
        
    def SetAlpha(self, alpha):
        self._alpha = float(alpha)
        
    def SetR(self, R):
        self._R = float(R)
    
    def __call__(self, u):
        return self._alpha*u*(1. - u/self._R)


def PlotSolution(timeArray, solutionArray, title):
    plt.plot(timeArray, 100*solutionArray, '-', linewidth=4) ## 100 --- for %
    plt.grid('on')
    plt.title(title)
    plt.xlabel('Время, с')
    plt.ylabel('Население, %')
    plt.show()

In [ ]:
rhs1 = LogisticRightHandSide()
rhs1.SetAlpha(0.2)
rhs1.SetR(1.)

Реализация функций для численного решения различными методами:

In [ ]:
def SolveViaForwardEuler(f, tEnd, numBlocks, initialValue):
    dt = float(tEnd)/numBlocks
    timeArray = np.linspace(0, tEnd, numBlocks + 1)
    solutionArray = np.zeros(numBlocks + 1)
    
    solutionArray[0] = initialValue
    # timestepping
    for i in range(numBlocks):
        solutionArray[i + 1] = solutionArray[i] + dt*f(solutionArray[i])
    return timeArray, solutionArray

# запуск расчета
timeArrayFE, solutionArrayFE = SolveViaForwardEuler(rhs1, tEnd, numBlocks, initialValue)


#############################################################################
def SolveViaPredictorCorrectorEuler(f, tEnd, numBlocks, initialValue):
    dt = float(tEnd)/numBlocks
    timeArray = np.linspace(0, tEnd, numBlocks + 1)
    solutionArray = np.zeros(numBlocks + 1)
    
    solutionArray[0] = initialValue
    # timesteping
    for i in range(numBlocks):
        uStar = solutionArray[i] + dt*f(solutionArray[i])
        solutionArray[i + 1] = solutionArray[i] + dt/2. \
                            * (f(solutionArray[i]) + f(uStar))
    
    return timeArray, solutionArray

                            
# запуск расчета
timeArrayPCE, solutionArrayPCE = SolveViaPredictorCorrectorEuler(rhs1, tEnd, numBlocks, initialValue)



###################################################################
def SolveViaRK4(f, tEnd, numBlocks, initialValue):
    dt = float(tEnd)/numBlocks
    timeArray = np.linspace(0, tEnd, numBlocks+1)
    solutionArray = np.zeros(numBlocks+1)
    
    solutionArray[0] = initialValue
    # timestepping
    for i in range(numBlocks):
        k1 = f(solutionArray[i])
        k2 = f(solutionArray[i] + dt/2.*k1)
        k3 = f(solutionArray[i] + dt/2.*k2)
        k4 = f(solutionArray[i] + dt*k3)
        
        solutionArray[i + 1] = solutionArray[i] + dt/6. * (k1 + 2.*k2 + 2.*k3 + k4)    
    
    return timeArray, solutionArray

# запуск расчета
timeArrayRK4, solutionArrayRK4 = SolveViaRK4(rhs1, tEnd, numBlocks, initialValue)

In [ ]:
# построение графиков численных решений
plt.figure(figsize=(11,7))
plt.title('Populations dynamics')
plt.plot(timeArrayFE, solutionArrayFE, label='Forward Euler')
plt.plot(timeArrayPCE, solutionArrayPCE, label='Predictor-Corrector Euler')
plt.plot(timeArrayRK4, solutionArrayRK4, label='Runge-Kutta 4')
plt.legend()
plt.xlabel('Time, s')
plt.ylabel('Population')
plt.grid('on')
plt.show()

### Объектно-ориентированный подход

Рассмотрим реализацию класса для проведения вычислений с помощью явного метода Эйлера. Класс __ForwardEuler__:

__Поля класса__:
* функция правой части ОДУ
* начальное условие
* множество точек сетки и ее параметры: кол-во точек, отрезков, шаг сетки
* массив для хранения значений численного решения
* начальный и конечный моменты времени
* ...(по Вашему усмотрению)

__Методы класса__:
* конструктор
* сеттеры для функции правой части, начального условия, параметров сетки 
* решить ОДУ (timestepping - цикл по точкам сетки)
* построить график численного решения
* ...(по Вашему усмотрению)

In [ ]:
import timeit

class ForwardEulerVersion1:
    
    def __init__(self):
        print('A blanc object of class ' + self.__class__.__name__ + ' is created.')
        
    def SetRightHandSide(self, f):
        self._f = f
    
    def SetInitialCondition(self, value):
        self._initialValue = value
       
    def SetGrid(self, numBlocks, tStart, tEnd):
        self._numBlocks, self._numPoints = numBlocks, numBlocks + 1
        self._dt = float(tEnd)/float(self._numBlocks)
        self._solutionArray = np.zeros(self._numPoints)
        self._timeArray = np.linspace(tStart, tEnd, self._numPoints)
        self._tStart, self._tEnd = tStart, tEnd
        
        # вспомогательная переменная, используется для вывода прогресса решения на экран
        self._isLessThanloo = True
        if self._numBlocks > 100:
            self._isLessThanloo = False
        
        
    
    def Solve(self):
        #self._counter = 0
        print('Start of timestepping via %s method...' % self.__class__.__name__)
        self._solutionArray[0] = self._initialValue
        
        start = timeit.default_timer()
        for i in range(self._numBlocks):
            
            # для краткости
            uOld, dt, f, t = self._solutionArray[i], self._dt, \
                             self._f, self._timeArray
                                
            # шаг явного метода Эйлера 
            self._solutionArray[i + 1] = uOld + dt*f(uOld)
            #self._counter += 1
            
            # каждые 100 шагов - вывод на экран
            if self._isLessThanloo or ((i + 1) % (self._numBlocks//100) == 0):
                print('Step #%d completed.' % (i + 1))
            
        end = float(timeit.default_timer() - start)
        print('Timestepping complete; calculations took %.2e s.' % end)
    
        return self._solutionArray, self._timeArray
    
    
    def PlotSolution(self):
        plt.figure(figsize=(11,7))
        plt.plot(self._timeArray, self._solutionArray, '-', linewidth=4)
        plt.grid('on')
        plt.title('Solution via %s method' % self.__class__.__name__)
        plt.xlabel('Time, s')
        plt.ylabel('Population')
        plt.show()

Функцию рассчета численного решения на неявном слое по времени можно оформить как отдельный метод класса __AdvanceNextStep__:

In [ ]:
class ForwardEulerVersion2:
    
    def __init__(self):
        print('A blanc object of class ' + self.__class__.__name__ + ' is created.')
        
    def SetRightHandSide(self, f):
        self._f = f
    
    def SetInitialCondition(self, value):
        self._initialValue = value
       
    def SetGrid(self, numBlocks, tStart, tEnd):
        self._numBlocks, self._numPoints = numBlocks, numBlocks + 1
        self._dt = float(tEnd)/float(self._numBlocks)
        self._solutionArray = np.zeros(self._numPoints)
        self._timeArray = np.linspace(tStart, tEnd, self._numPoints)
        self._tStart, self._tEnd = tStart, tEnd
        
        # вспомогательная переменная, используется для вывода прогресса решения на экран
        self._isLessThanloo = True
        if self._numBlocks > 100:
            self._isLessThanloo = False
        
        
    def Solve(self):
        print('Start of timestepping via %s method...' % self.__class__.__name__)
        self._solutionArray[0] = self._initialValue
        
        start = timeit.default_timer()
        for i in range(self._numBlocks):
            self._solutionArray[i + 1] = self.AdvanceNextStep(i)
            
            # каждые 100 шагов - вывод на экран
            if self._isLessThanloo or ((i + 1) % (self._numBlocks//100) == 0):
                print('Step #%d completed.' % (i + 1))
            
        end = float(timeit.default_timer() - start)
        print('Timestepping complete; calculations took %.2e s.' % end)
    
        return self._solutionArray, self._timeArray
    
    
    def AdvanceNextStep(self, i):
        # для краткости
        uOld, dt, f = self._solutionArray[i], self._dt, self._f,  
        # шаг явного метода Эйлера
        uNew = uOld + dt*f(uOld)
        
        return uNew
        
    
    def PlotSolution(self):
        plt.figure(figsize=(11,7))
        plt.plot(self._timeArray, self._solutionArray, '-', linewidth=4)
        plt.grid('on')
        plt.title('Solution via %s method' % self.__class__.__name__)
        plt.xlabel('Time, s')
        plt.ylabel('Population')
        plt.show()

Теперь протестируем последнюю версию класса-солвера на логистическом уравнении. Говорящие имена методов класса позволяют стороннему пользователю/разработчику разобраться в коде модуля, в котором обычно запускают расчеты:

In [ ]:
listOfMethods = [ForwardEulerVersion2()]

for method in listOfMethods:
    method.SetRightHandSide(rhs1)
    method.SetInitialCondition(0.1)
    method.SetGrid(numBlocks=200, tStart=0., tEnd=40.)
    method.Solve()
    method.PlotSolution()

В задании потребуется написать классы для одношаговых методов численного решения ОДУ: несколько явных и 1 неявный (метод трапеций 2-го порядка точности).

## Примечания по реализации неявного метода трапеций
Формула для проведения вычислений:

$$ u_{n+1} = u_n + \frac{\Delta t}{2} \left( f(u_n, t_n) + f(u_{n+1}, t_{n+1}) \right), \quad n = 0,...,N $$ 

Для решения получившегося нелинейного уравнения относительно $u_{n+1}$ использовать итерационный метод Ньютона:

$$ F(u_{n+1}) \equiv  u_{n+1} - \frac{\Delta t}{2} f(u_{n+1}, t_{n+1}) - u_n - \frac{\Delta t}{2} f(u_n, t_n) = 0$$

$$ u_{n+1}^{(k+1)} = u_{n+1}^{(k)} - \frac{F( u_{n+1}^{(k)} )}{ F'(u_{n+1}^{(k)}) } $$ 

$$
k = 0,...,K \quad   \Vert u_{n+1}^{(K)} - u_{n+1}^{(K-1)} \Vert \leq \varepsilon
$$

Для ускорения сходимости начальное приближение рассчитывать по формуле явного метода Эйлера:

$$ u_{n+1}^{(0)} = u_n + \Delta t  f(u_n, t_n) $$

Для приближенного вычисления производной использовать формулу центральной разности 2-го порядка точности:

$$
    F'(u_{n+1}^{(k)}) \approx F'_{num}(u_{n+1}^{(k)}) = \frac{F(u_{n+1}^{(k)} + h) - F(u_{n+1}^{(k)} - h)}{2h}
$$

Решение нелинейного уравнения с помощью метода Ньютона можно оформить как внутри класса, так и в виде отдельной функции. При вычислении производной задействовать функторы, реализованные в __Задаче 2__.

## Вопросы?